# Instantiate a new SageMaker Model

In [15]:
import json
from sagemaker.xgboost.estimator import XGBoost

In [2]:
import xgboost as xgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
import joblib

# Load sample dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define XGBoost classifier
clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Train the model
clf.fit(X_train, y_train)

# Save the model to disk
joblib.dump(clf, 'xgboost_model.pkl')

# Load the model from disk
loaded_model = joblib.load('xgboost_model.pkl')

# Use the loaded model for predictions
predictions = loaded_model.predict(X_test)

# Print predictions
print(predictions)

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


In [12]:
loaded_model.predict().tolist()

[1,
 0,
 2,
 1,
 1,
 0,
 1,
 2,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 2,
 1,
 1,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 0]

In [56]:
!tar czvf model.tar.gz eba_xgboost_model.pkl

eba_xgboost_model.pkl


In [58]:
!aws s3 cp model.tar.gz s3://sagemaker-us-east-1-471112729843/sagemaker/regions-EBA/171/

upload: ./model.tar.gz to s3://sagemaker-us-east-1-471112729843/sagemaker/regions-EBA/171/model.tar.gz


In [59]:
json.dumps({"inputs": {"data": list(X_test.tolist())}})

'{"inputs": {"data": [[6.1, 2.8, 4.7, 1.2], [5.7, 3.8, 1.7, 0.3], [7.7, 2.6, 6.9, 2.3], [6.0, 2.9, 4.5, 1.5], [6.8, 2.8, 4.8, 1.4], [5.4, 3.4, 1.5, 0.4], [5.6, 2.9, 3.6, 1.3], [6.9, 3.1, 5.1, 2.3], [6.2, 2.2, 4.5, 1.5], [5.8, 2.7, 3.9, 1.2], [6.5, 3.2, 5.1, 2.0], [4.8, 3.0, 1.4, 0.1], [5.5, 3.5, 1.3, 0.2], [4.9, 3.1, 1.5, 0.1], [5.1, 3.8, 1.5, 0.3], [6.3, 3.3, 4.7, 1.6], [6.5, 3.0, 5.8, 2.2], [5.6, 2.5, 3.9, 1.1], [5.7, 2.8, 4.5, 1.3], [6.4, 2.8, 5.6, 2.2], [4.7, 3.2, 1.6, 0.2], [6.1, 3.0, 4.9, 1.8], [5.0, 3.4, 1.6, 0.4], [6.4, 2.8, 5.6, 2.1], [7.9, 3.8, 6.4, 2.0], [6.7, 3.0, 5.2, 2.3], [6.7, 2.5, 5.8, 1.8], [6.8, 3.2, 5.9, 2.3], [4.8, 3.0, 1.4, 0.3], [4.8, 3.1, 1.6, 0.2]]}}'

## Instantiate a new Model

In [68]:
import boto3
import sagemaker
from sagemaker.xgboost.model import XGBoostModel

In [84]:
sm_client = boto3.client('sagemaker', region_name="us-east-1")
role = "arn:aws:iam::471112729843:role/SageMakerEMR-Development-OSE-EMR-SageMakerExecutionRole"

In [85]:
xgboost_model = XGBoostModel(
    model_data="s3://sagemaker-us-east-1-471112729843/sagemaker/regions-EBA/171/model.tar.gz",
    role=role,
    entry_point="inference.py",
    framework_version="1.5-1"
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [92]:
model_package_group_name = "iris-eba-xgboost"
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "Sample model package group"
}

create_model_package_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
print('ModelPackageGroup Arn : {}'.format(create_model_package_group_response['ModelPackageGroupArn']))

ClientError: An error occurred (ValidationException) when calling the CreateModelPackageGroup operation: Model Package Group already exists: arn:aws:sagemaker:us-east-1:471112729843:model-package-group/iris-eba-xgboost

In [93]:
xgboost_model.register(
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=["ml.c5.2xlarge"],
    transform_instances=["ml.c5.2xlarge"],
    model_package_group_name="iris-eba-xgboost",
    description="dummy model package group",
    framework="XGBOOST",
    nearest_model_name="XGBoost",
    approval_status="Approved"
)

In [94]:
model_package_arn = "arn:aws:sagemaker:us-east-1:471112729843:model-package/iris-eba-xgboost/6"
sm_client.describe_model_package(ModelPackageName=model_package_arn)

{'ModelPackageGroupName': 'iris-eba-xgboost',
 'ModelPackageVersion': 6,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:471112729843:model-package/iris-eba-xgboost/6',
 'ModelPackageDescription': 'dummy model package group',
 'CreationTime': datetime.datetime(2024, 1, 23, 18, 52, 5, 59000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1',
    'ImageDigest': 'sha256:46b359a986d43b894d0334104e0836bdb12c753f71f50a6f13ad81d44cfcd3a1',
    'ModelDataUrl': 's3://sagemaker-us-east-1-471112729843/sagemaker/regions-EBA/171/model.tar.gz',
    'Environment': {'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
     'SAGEMAKER_PROGRAM': 'inference.py',
     'SAGEMAKER_REGION': 'us-east-1',
     'SAGEMAKER_SUBMIT_DIRECTORY': 's3://sagemaker-us-east-1-471112729843/sagemaker-xgboost-2024-01-23-18-52-04-279/sourcedir.tar.gz'},
    'Framework': 'XGBOOST',
    'NearestModelName': 'XGBoost'}],
  'SupportedTransformInstance

In [95]:
model_name = "iris-eba-xgboost-serverless-model-151-fix" 
print("Model name : {}".format(model_name))
container_list = [{"ModelPackageName": model_package_arn}]

create_model_response = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, Containers=container_list
)
print("Model arn : {}".format(create_model_response["ModelArn"]))

Model name : iris-eba-xgboost-serverless-model-151-fix
Model arn : arn:aws:sagemaker:us-east-1:471112729843:model/iris-eba-xgboost-serverless-model-151-fix


In [96]:
endpoint_config_name = "iris-eba-xgboost-serverless-model-151-fix-epconfig" 
print(endpoint_config_name)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "ServerlessConfig": {"MemorySizeInMB": 1024, "MaxConcurrency": 10},
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)
print("Endpoint Configuration Arn: " + create_endpoint_config_response["EndpointConfigArn"])

iris-eba-xgboost-serverless-model-151-fix-epconfig
Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:471112729843:endpoint-config/iris-eba-xgboost-serverless-model-151-fix-epconfig


In [97]:
endpoint_name = "iris-eba-xgboost-serverless-model-151-ep-fix" 
print("EndpointName={}".format(endpoint_name))

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

EndpointName=iris-eba-xgboost-serverless-model-151-ep-fix
arn:aws:sagemaker:us-east-1:471112729843:endpoint/iris-eba-xgboost-serverless-model-151-ep-fix


## Local Mode

In [79]:
from sagemaker.local import LocalSession

In [82]:
xgb_inference_model = XGBoostModel(
    model_data="file://model.tar.gz",
    role=role,
    entry_point="inference.py",
    framework_version="1.7-1",
    sagemaker_session=LocalSession()
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [83]:
print('Deploying endpoint in local mode')
predictor = xgb_inference_model.deploy(
    initial_instance_count=1,
    instance_type="local",
)

Deploying endpoint in local mode


FileNotFoundError: [Errno 2] No such file or directory: 'docker'